In [1]:
!pip install --upgrade pip -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.5 MB 1.4 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.2
    Uninstalling pip-20.2.2:
      Successfully uninstalled pip-20.2.2


In [2]:
!pip install tensorflow==2.3.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 320.5 MB 1.1 kB/s  eta 0:00:01 eta 0:01:49B/s eta 0:01:4813.7 MB 2.9 MB/s eta 0:01:4615██▉                         | 67.9 MB 307 kB/s eta 0:13:43            | 69.6 MB 306 kB/s eta 0:13:38     |██████████▉                     | 108.5 MB 4.0 MB/s eta 0:00:53      | 113.3 MB 4.0 MB/s eta 0:00:52120 kB/s eta 0:25:06███▎                 | 143.1 MB 120 kB/s eta 0:24:31/s eta 0:00:02   |██████████████████▌             | 184.7 MB 139 kB/s eta 0:16:11:00:03ta 0:03:18
     |████████████████████████████████| 20.6 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 26.0 MB 353 kB/s eta 0:00:011
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5

In [3]:
!pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 829 kB 21 kB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=44a18ab2d7866a42e48c535561ad110203c59593e03e2a7aa56302e211523f52
  Stored in directory: /home/kesci/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future


In [4]:
!pip install opencv-python -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 49.5 MB 1.5 kB/s  eta 0:00:01:01:54███████████████████████▋      | 39.6 MB 129 kB/s eta 0:01:17 MB 49.6 MB/s eta 0:00:01


In [5]:
!pip install numba -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [6]:
!pip install efficientnet -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 50 kB 2.1 MB/s eta 0:00:011


# DataGenerator

## train

In [1]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from keras.utils import Sequence
import cv2
import glob
import os
from tqdm import tqdm

class DataGenerator_train(Sequence):
    """
    基于Sequence的自定义Keras数据生成器
    """
    def __init__(self, filepath, batch_size=8, imgshape=(256, 472),
                 n_channels=3, n_classes=13, label_dic={'others':0,'sunny':1,'cloudy':2},
                 label_all={},shuffle=True):
        """ 初始化方法
        :param filepath :数据文件地址
        :param batch_size: batch size
        :param imgshape: 图像大小
        :param n_channels: 图像通道
        :param n_classes: 标签类别
        :param shuffle: 每一个epoch后是否打乱数据
        """
        self.filepath=filepath
        # 文件地址list
        self.pathlist=os.listdir(self.filepath)
        self.batch_size = batch_size
        self.imgshape = imgshape
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.label_dic = label_dic
        self.label_all = label_all
        # 每个epoch之后更新索引
        self.on_epoch_end()

    def __getitem__(self, index):
        """生成每一批次训练数据
        :param index: 批次索引
        :return: 训练图像和标签
        """
        # 生成批次索引
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # 索引列表
        batch_pathlist = [self.pathlist[k] for k in indexes]
        # 生成数据
        X = self._generate_X(batch_pathlist)
        y = self._generate_y(batch_pathlist,self.label_all,self.label_dic)
        return X, y

    def __len__(self):
        """每个epoch下的批次数量,也就是每个epoch的iteration
        """
        return int(np.floor(len(self.pathlist) / self.batch_size))

    def _generate_X(self, batch_pathlist):
        """生成每一批次的图像
        :param list_IDs_temp: 批次数据索引列表
        :return: 一个批次的图像
        """
        # 初始化
        X = np.empty((self.batch_size, *self.imgshape, self.n_channels))
        # 生成数据
        for i, path in enumerate(batch_pathlist):
            # 存储一个批次
            X[i,] = self._load_image(self.filepath+path)
        return X

    def _generate_y(self, batch_pathlist,label_all,label_dic):
        """生成每一批次的标签
        :param list_IDs_temp: 批次数据索引列表
        :return: 一个批次的标签
        """
        y = np.empty((self.batch_size, ), dtype=int)
        # Generate data
        for i, path in enumerate(batch_pathlist):
            y[i] = label_dic[label_all[path]]
        y = np.eye(3)[y-1]
        return y

    def on_epoch_end(self):
        """每个epoch之后更新索引
        """
        self.indexes = np.arange(len(self.pathlist))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def _load_image(self, image_path):
        """cv2读取图像
        """
        # img = cv2.imread(image_path)
        img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
        w, h, _ = img.shape
        if w>h:
            img = np.rot90(img)
        img = cv2.resize(img, self.imgshape)
        return img


In [3]:
with open('/home/kesci/data1/competition/train_set/Part1.json', encoding="utf-8") as f:
    label_all = json.load(f)
print('label load done')
# Parameters
params = {'batch_size': 10,
        'n_classes': 3,
        'n_channels': 3,
        'shuffle': True,
        'label_dic':{'others':0,'sunny':1,'cloudy':2},
        'label_all':label_all,
        'imgshape':(256,256)}
train_filepath=r'/home/kesci/data1/competition/train_set/Part1/'
# Generators
training_generator = DataGenerator_train(train_filepath, **params)
print('training_generator build done')
# for x,y in training_generator:
#     print(y)

label load done
training_generator build done


## test

In [2]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from keras.utils import Sequence
import cv2
import glob
import os
from tqdm import tqdm

class DataGenerator_test(Sequence):
    """
    基于Sequence的自定义Keras数据生成器
    """
    def __init__(self, filepath, batch_size=8, imgshape=(256, 472),
                 n_channels=3, n_classes=13,shuffle=True):
        """ 初始化方法
        :param filepath :数据文件地址
        :param batch_size: batch size
        :param imgshape: 图像大小
        :param n_channels: 图像通道
        :param n_classes: 标签类别
        :param shuffle: 每一个epoch后是否打乱数据
        """
        self.filepath=filepath
        # 文件地址list
        self.pathlist=os.listdir(self.filepath)
        self.batch_size = batch_size
        self.imgshape = imgshape
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.test_ID_list = []
        # 每个epoch之后更新索引
        self.on_epoch_end()

    def __getitem__(self, index):
        """生成每一批次训练数据
        :param index: 批次索引
        :return: 训练图像和标签
        """
        # 生成批次索引
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # 索引列表
        batch_pathlist = [self.pathlist[k] for k in indexes]
        # 生成数据
        X = self._generate_X(batch_pathlist)
        self.test_ID_list.append(batch_pathlist)
        return X

    def __len__(self):
        """每个epoch下的批次数量,也就是每个epoch的iteration
        """
        return int(np.floor(len(self.pathlist) / self.batch_size))

    def _generate_X(self, batch_pathlist):
        """生成每一批次的图像
        :param list_IDs_temp: 批次数据索引列表
        :return: 一个批次的图像
        """
        # 初始化
        X = np.empty((self.batch_size, *self.imgshape, self.n_channels))
        # 生成数据
        for i, path in enumerate(batch_pathlist):
            # 存储一个批次
            X[i,] = self._load_image(self.filepath+path)
        return X

    def on_epoch_end(self):
        """每个epoch之后更新索引
        """
        self.indexes = np.arange(len(self.pathlist))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def _load_image(self, image_path):
        """cv2读取图像
        """
        # img = cv2.imread(image_path)
        img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
        w, h, _ = img.shape
        if w>h:
            img = np.rot90(img)
        img = cv2.resize(img, self.imgshape)
        return img


In [4]:
# Parameters
params = {'batch_size': 15,
        'n_classes': 3,
        'n_channels': 3,
        'shuffle': True,
        'imgshape':(256,256)}
test_filepath=r'/home/kesci/data1/competition/train_set/Part1/'
# Generators
testing_generator = DataGenerator_test(test_filepath, **params)
print('testing_generator build done')
# for x in testing_generator:
#     print('aaa')

# testing_generator.test_ID_list



testing_generator build done


# CNN BaseLine

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

from tensorflow.keras.callbacks import EarlyStopping

In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
earlystop_callback = EarlyStopping(monitor='accuracy',
                                    min_delta=0.00001,
                                    patience=20,
                                    verbose=1,
                                    mode='max',
                                    baseline=None,
                                    restore_best_weights=True,)

model.fit_generator(generator=training_generator,
                # validation_split=0.3, 
                epochs=50,
                callbacks=[earlystop_callback])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
 1759/11305 [===>..........................] - ETA: 20:00 - loss: 6.2017 - accuracy: 0.4409

In [ ]:
model.predict_generator(generator=testing_generator,verbose=1)
sub_ID = testing_generator.test_ID_list

# EfficientNet

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

from tensorflow.keras.callbacks import EarlyStopping

In [5]:
import tensorflow as tf
import math

NUM_CLASSES = 3

def swish(x):
    return x * tf.nn.sigmoid(x)


def round_filters(filters, multiplier):
    depth_divisor = 8
    min_depth = None
    min_depth = min_depth or depth_divisor
    filters = filters * multiplier
    new_filters = max(min_depth, int(filters + depth_divisor / 2) // depth_divisor * depth_divisor)
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)


def round_repeats(repeats, multiplier):
    if not multiplier:
        return repeats
    return int(math.ceil(multiplier * repeats))


class SEBlock(tf.keras.layers.Layer):
    def __init__(self, input_channels, ratio=0.25):
        super(SEBlock, self).__init__()
        self.num_reduced_filters = max(1, int(input_channels * ratio))
        self.pool = tf.keras.layers.GlobalAveragePooling2D()
        self.reduce_conv = tf.keras.layers.Conv2D(filters=self.num_reduced_filters,
                                                  kernel_size=(1, 1),
                                                  strides=1,
                                                  padding="same")
        self.expand_conv = tf.keras.layers.Conv2D(filters=input_channels,
                                                  kernel_size=(1, 1),
                                                  strides=1,
                                                  padding="same")

    def call(self, inputs, **kwargs):
        branch = self.pool(inputs)
        branch = tf.expand_dims(input=branch, axis=1)
        branch = tf.expand_dims(input=branch, axis=1)
        branch = self.reduce_conv(branch)
        branch = swish(branch)
        branch = self.expand_conv(branch)
        branch = tf.nn.sigmoid(branch)
        output = inputs * branch
        return output


class MBConv(tf.keras.layers.Layer):
    def __init__(self, in_channels, out_channels, expansion_factor, stride, k, drop_connect_rate):
        super(MBConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.drop_connect_rate = drop_connect_rate
        self.conv1 = tf.keras.layers.Conv2D(filters=in_channels * expansion_factor,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.dwconv = tf.keras.layers.DepthwiseConv2D(kernel_size=(k, k),
                                                      strides=stride,
                                                      padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.se = SEBlock(input_channels=in_channels * expansion_factor)
        self.conv2 = tf.keras.layers.Conv2D(filters=out_channels,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(rate=drop_connect_rate)

    def call(self, inputs, training=None, **kwargs):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = swish(x)
        x = self.dwconv(x)
        x = self.bn2(x, training=training)
        x = self.se(x)
        x = swish(x)
        x = self.conv2(x)
        x = self.bn3(x, training=training)
        if self.stride == 1 and self.in_channels == self.out_channels:
            if self.drop_connect_rate:
                x = self.dropout(x, training=training)
            x = tf.keras.layers.add([x, inputs])
        return x


def build_mbconv_block(in_channels, out_channels, layers, stride, expansion_factor, k, drop_connect_rate):
    block = tf.keras.Sequential()
    for i in range(layers):
        if i == 0:
            block.add(MBConv(in_channels=in_channels,
                             out_channels=out_channels,
                             expansion_factor=expansion_factor,
                             stride=stride,
                             k=k,
                             drop_connect_rate=drop_connect_rate))
        else:
            block.add(MBConv(in_channels=out_channels,
                             out_channels=out_channels,
                             expansion_factor=expansion_factor,
                             stride=1,
                             k=k,
                             drop_connect_rate=drop_connect_rate))
    return block


class EfficientNet(tf.keras.Model):
    def __init__(self, width_coefficient, depth_coefficient, dropout_rate, drop_connect_rate=0.2):
        super(EfficientNet, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(filters=round_filters(32, width_coefficient),
                                            kernel_size=(3, 3),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.block1 = build_mbconv_block(in_channels=round_filters(32, width_coefficient),
                                         out_channels=round_filters(16, width_coefficient),
                                         layers=round_repeats(1, depth_coefficient),
                                         stride=1,
                                         expansion_factor=1, k=3, drop_connect_rate=drop_connect_rate)
        self.block2 = build_mbconv_block(in_channels=round_filters(16, width_coefficient),
                                         out_channels=round_filters(24, width_coefficient),
                                         layers=round_repeats(2, depth_coefficient),
                                         stride=2,
                                         expansion_factor=6, k=3, drop_connect_rate=drop_connect_rate)
        self.block3 = build_mbconv_block(in_channels=round_filters(24, width_coefficient),
                                         out_channels=round_filters(40, width_coefficient),
                                         layers=round_repeats(2, depth_coefficient),
                                         stride=2,
                                         expansion_factor=6, k=5, drop_connect_rate=drop_connect_rate)
        self.block4 = build_mbconv_block(in_channels=round_filters(40, width_coefficient),
                                         out_channels=round_filters(80, width_coefficient),
                                         layers=round_repeats(3, depth_coefficient),
                                         stride=2,
                                         expansion_factor=6, k=3, drop_connect_rate=drop_connect_rate)
        self.block5 = build_mbconv_block(in_channels=round_filters(80, width_coefficient),
                                         out_channels=round_filters(112, width_coefficient),
                                         layers=round_repeats(3, depth_coefficient),
                                         stride=1,
                                         expansion_factor=6, k=5, drop_connect_rate=drop_connect_rate)
        self.block6 = build_mbconv_block(in_channels=round_filters(112, width_coefficient),
                                         out_channels=round_filters(192, width_coefficient),
                                         layers=round_repeats(4, depth_coefficient),
                                         stride=2,
                                         expansion_factor=6, k=5, drop_connect_rate=drop_connect_rate)
        self.block7 = build_mbconv_block(in_channels=round_filters(192, width_coefficient),
                                         out_channels=round_filters(320, width_coefficient),
                                         layers=round_repeats(1, depth_coefficient),
                                         stride=1,
                                         expansion_factor=6, k=3, drop_connect_rate=drop_connect_rate)

        self.conv2 = tf.keras.layers.Conv2D(filters=round_filters(1280, width_coefficient),
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.pool = tf.keras.layers.GlobalAveragePooling2D()
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)
        self.fc = tf.keras.layers.Dense(units=NUM_CLASSES,
                                        activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = swish(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = swish(x)
        x = self.pool(x)
        x = self.dropout(x, training=training)
        x = self.fc(x)

        return x


def get_efficient_net(width_coefficient, depth_coefficient, resolution, dropout_rate):
    net = EfficientNet(width_coefficient=width_coefficient,
                       depth_coefficient=depth_coefficient,
                       dropout_rate=dropout_rate)
    net.build(input_shape=(None, resolution, resolution, 3))
    net.summary()

    return net


def efficient_net_b0():
    return get_efficient_net(1.0, 1.0, 224, 0.2)


def efficient_net_b1():
    return get_efficient_net(1.0, 1.1, 240, 0.2)


def efficient_net_b2():
    return get_efficient_net(1.1, 1.2, 260, 0.3)


def efficient_net_b3():
    return get_efficient_net(1.2, 1.4, 300, 0.3)


def efficient_net_b4():
    return get_efficient_net(1.4, 1.8, 380, 0.4)


def efficient_net_b5():
    return get_efficient_net(1.6, 2.2, 456, 0.4)


def efficient_net_b6():
    return get_efficient_net(1.8, 2.6, 528, 0.5)


def efficient_net_b7():
    return get_efficient_net(2.0, 3.1, 600, 0.5)

In [6]:
width_coefficient = 3
depth_coefficient = 3
resolution = 3
dropout_rate = 0.5
model = get_efficient_net(width_coefficient, depth_coefficient, resolution, dropout_rate)

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
earlystop_callback = EarlyStopping(monitor='accuracy',
                                    min_delta=0.00001,
                                    patience=20,
                                    verbose=1,
                                    mode='max',
                                    baseline=None,
                                    restore_best_weights=True,)

model.fit_generator(generator=training_generator,
                # validation_split=0.3, 
                epochs=5,
                callbacks=[earlystop_callback])

In [ ]:
model.predict_generator(generator=testing_generator,verbose=1)
sub_ID = testing_generator.test_ID_list

# EfficientNet Pre-train

In [5]:
import efficientnet.keras as efn 
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

model_pre = efn.EfficientNetB7(weights='/home/kesci/efficientnet-b7_noisy-student.h5')  # or weights='imagenet'
model_pre.summary()
# for layer in model_pre.layers:
#         if layer.name != 'probs':
#             layer.trainable = False

Model: "efficientnet-b7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 600, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 300, 300, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 300, 300, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 300, 300, 64) 0           stem_bn[0][0]                    
____________________________________________________________________________________

In [6]:
from tensorflow.keras.models import Model, Sequential

model = Sequential()
model.add(model_pre)
model.add(Dense(3, activation='softmax'))


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 1000)              66658680  
_________________________________________________________________
dense (Dense)                (None, 3)                 3003      
Total params: 66,661,683
Trainable params: 66,350,963
Non-trainable params: 310,720
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
              

In [101]:
earlystop_callback = EarlyStopping(monitor='accuracy',
                                    min_delta=0.00001,
                                    patience=20,
                                    verbose=1,
                                    mode='max',
                                    baseline=None,
                                    restore_best_weights=True,)
 
model.fit_generator(generator=training_generator,
                # validation_split=0.3, 
                epochs=50,
                callbacks=[earlystop_callback])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
 6303/16958 [==========>...................] - ETA: 2:05:05 - loss: 0.9349 - accuracy: 0.5878

In [ ]:
model.save('trained_model.h5')

# train on P2-P4

In [ ]:
model = load_model('trained_model.h5')

In [ ]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from keras.utils import Sequence
import cv2
import glob
import os
from tqdm import tqdm

class DataGenerator_train(Sequence):
    """
    基于Sequence的自定义Keras数据生成器
    """
    def __init__(self, filepath, batch_size=8, imgshape=(256, 472),
                 n_channels=3, n_classes=13, label_dic={'others':0,'sunny':1,'cloudy':2},
                 label_all={},shuffle=True):
        """ 初始化方法
        :param filepath :数据文件地址
        :param batch_size: batch size
        :param imgshape: 图像大小
        :param n_channels: 图像通道
        :param n_classes: 标签类别
        :param shuffle: 每一个epoch后是否打乱数据
        """
        self.filepath=filepath
        # 文件地址list
        self.pathlist=os.listdir(self.filepath)
        self.batch_size = batch_size
        self.imgshape = imgshape
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.label_dic = label_dic
        self.label_all = label_all
        # 每个epoch之后更新索引
        self.on_epoch_end()

    def __getitem__(self, index):
        """生成每一批次训练数据
        :param index: 批次索引
        :return: 训练图像和标签
        """
        # 生成批次索引
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # 索引列表
        batch_pathlist = [self.pathlist[k] for k in indexes]
        # 生成数据
        X = self._generate_X(batch_pathlist)
        y = self._generate_y(batch_pathlist,self.label_all,self.label_dic)
        return X, y

    def __len__(self):
        """每个epoch下的批次数量,也就是每个epoch的iteration
        """
        return int(np.floor(len(self.pathlist) / self.batch_size))

    def _generate_X(self, batch_pathlist):
        """生成每一批次的图像
        :param list_IDs_temp: 批次数据索引列表
        :return: 一个批次的图像
        """
        # 初始化
        X = np.empty((self.batch_size, *self.imgshape, self.n_channels))
        # 生成数据
        for i, path in enumerate(batch_pathlist):
            # 存储一个批次
            X[i,] = self._load_image(self.filepath+path)
        return X

    def _generate_y(self, batch_pathlist,label_all,label_dic):
        """生成每一批次的标签
        :param list_IDs_temp: 批次数据索引列表
        :return: 一个批次的标签
        """
        y = np.empty((self.batch_size, ), dtype=int)
        # Generate data
        for i, path in enumerate(batch_pathlist):
            y[i] = label_dic[label_all[path]]
        y = np.eye(3)[y-1]
        return y

    def on_epoch_end(self):
        """每个epoch之后更新索引
        """
        self.indexes = np.arange(len(self.pathlist))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def _load_image(self, image_path):
        """cv2读取图像
        """
        # img = cv2.imread(image_path)
        img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
        w, h, _ = img.shape
        if w>h:
            img = np.rot90(img)
        img = cv2.resize(img, self.imgshape)
        return img


In [ ]:
with open('/home/kesci/data1/competition/train_set/Part1.json', encoding="utf-8") as f:
    label_all = json.load(f)
print('label load done')
# Parameters
params = {'batch_size': 10,
        'n_classes': 3,
        'n_channels': 3,
        'shuffle': True,
        'label_dic':{'others':0,'sunny':1,'cloudy':2},
        'label_all':label_all,
        'imgshape':(256,256)}
train_filepath=r'/home/kesci/data1/competition/train_set/Part1/'
# Generators
training_generator = DataGenerator_train(train_filepath, **params)
print('training_generator build done')
# for x,y in training_generator:
#     print(y)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
              

In [ ]:
earlystop_callback = EarlyStopping(monitor='accuracy',
                                    min_delta=0.00001,
                                    patience=20,
                                    verbose=1,
                                    mode='max',
                                    baseline=None,
                                    restore_best_weights=True,)
 
model.fit_generator(generator=training_generator,
                # validation_split=0.3, 
                epochs=50,
                callbacks=[earlystop_callback])

In [ ]:
model.save('trained_model.h5')